In [1]:
import pandas as pd
%matplotlib inline
import haversine as hv
import numpy as np
from ipywidgets import (widgets, HBox, VBox, Layout, interact)
import bqplot.pyplot as plt
import us
from matplotlib.pyplot import imshow
import matplotlib.pyplot as mpl

In [2]:
# Read in 
location = pd.read_csv("/home/mavis22/work/data-readonly/transportable_array/location.txt", delimiter="\t", names=["longitude","latitude","a","b"])
del location["a"], location["b"]
# From examples_week10.ipynb (IS590DV Class example)

In [3]:
# Tohoku : 142.369, 38.322
Tohoku=(142.369, 38.322)

In [4]:
# Calculate the distances
distance=[]
for row in location.iterrows():
    station = (row[1][0],row[1][1])
    distance.append(hv.haversine(Tohoku,station))

In [5]:
# Add distance column to location DataFrame
location["distance"]=distance

In [6]:
location["order"]=location.index

In [7]:
# Sort by distacnes (acsending)
location = location.sort_values(by=['distance'])

In [8]:
order=list(location["order"])

In [9]:
# Read in detector data
detector=pd.read_csv("/home/mavis22/work/data-readonly/transportable_array/data_tohoku_norm_transpose.csv",names=np.arange(0,438), index_col=False)

In [10]:
ordered_detector = detector[order[0]]
for each in order[1:]:
    ordered_detector = pd.concat([ordered_detector,detector[each]],axis=1)

In [11]:
# Add time range column, from week 10 example
time = pd.date_range("2:46PM", "6:46PM", freq="1s")
time -= time[0]
ordered_detector["time"] = time
ordered_detector.set_index("time", inplace=True)

In [12]:
od = np.log(ordered_detector.values**2)
od = od.transpose()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [13]:
from __future__ import print_function
from bqplot import *

In [14]:
# Custom Time Slider
time = widgets.IntSlider(min=0,max=14400, step=1, readout=False)
def out(time):
    hour = time//3600
    time=time-hour*3600
    minute = time//60
    time=time-minute*60
    second = time
    if hour<10:
        hour='0'+str(hour)
    else:
        hour=str(hour)
    if minute<10:
        minute='0'+str(minute)
    else:
        minute=str(minute)
    if second<10:
        second='0'+str(second)
    else:
        second=str(second)
  
    output=hour+":"+minute+":"+second
    print(output)

time_out = widgets.interactive_output(out, {'time':time})

time_slider = widgets.HBox([time, time_out])

def update_thing(event):
    print(event)

In [15]:
time_slider

HBox(children=(IntSlider(value=0, max=14400, readout=False), Output(outputs=({'output_type': 'stream', 'text': '00:00:00\n', 'name': 'stdout'},))))

In [16]:
@interact(Time = widgets.IntSlider(min=0,max=14400, step=1, readout=True))
def make_plot(Time):
    time_slider=widgets.HBox([Time, time_out])
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    # plt.clear()
    
    # map_res = Map(map_data=topo_load('map_data/USStatesMap.json'))
    scatter_chart = plt.scatter(x=location['longitude'], y=location['latitude'],
                                colors = ['White'],
                           scales= {'color': ColorScale(colors=['White','Red'])},
                               color=detector.iloc[Time])
    scatter = Figure(marks=[scatter_chart], title='Detectors in U.S.', 
                     fig_margin={'top':50,'bottom':30, 'left':10, 'right':-30})
    
    
    time_x=LinearScale()
    wave_y=LinearScale()
    ax_x=Axis(scale=time_x)
    ax_y=Axis(scale=wave_y,orientation='vertical', label='')
    line_marks = Lines(x=[], y=[], scales={'x':time_x, 'y':wave_y})
    selected=detector[0]
    
    line_marks.x, line_marks.y = np.arange(0,14400,1), selected
    ax_x.label, ax_y.label ='Time', 'Magnitude'
    
    
    
    line_marks.visible = True
    lineplot = Figure(marks = [line_marks], fig_margin={'top':50, 'bottom':50, 'left':70, 'right':20})
    lineplot.axes = [ax_x, ax_y]
    lineplot.title='Waveform of Detector No.0'
    
    spect = mpl.imshow(od, aspect='auto', cmap='viridis',origin='lower')
    line1 = mpl.axvline(Time, lw=4, color='r')
    line2 = mpl.axhline(10, lw = 4, color='r')
    mpl.axis('off')
    
    
    def get_target(self, target):
        if target['data'] != None:
            
                mpl.clf()
                
                selected=detector[target['data']['index']]
                lineplot.title='Waveform of Detector No.{0}'.format(target['data']['index'])
                line_marks.x, line_marks.y = np.arange(0,14400,1), selected
                ax_x.label, ax_y.label ='Time', 'Magnitude'
                lineplot.axes = [ax_x, ax_y]
                
                
                spect = imshow(od, aspect='auto', cmap='viridis',origin='lower')
                line1 = mpl.axvline(Time, lw=4, color='r')
                line2 = mpl.axhline(target['data']['index'], lw = 4, color='r')
                # line2.set_data([-10, 15000], [target['data']['index'],target['data']['index']])
                mpl.axis('off')
            
                line_marks.visible = True
                
                # ax.plot([-100,15000],[target['data']['index'], target['data']['index']])
                # fig.colorbar(im, ax=ax, shrink=0.5)

        else:#when click on the background, lineplot will not be shown
                ax_y.label, ax_x.lable, lineplot.title = '', '', ''
                lineplot.axes = []
                line_marks.visible = False
        return(target['data']['index']) # Target is  dict type

    scatter_chart.on_element_click(get_target)
    
    return VBox([scatter,lineplot])
    
    

interactive(children=(IntSlider(value=0, description='Time', max=14400), Output()), _dom_classes=('widget-interact',))